In [5]:
import pandas as pd      
import geopandas as gpd   
import osmnx as ox   
from pathlib import Path

import matplotlib.pyplot as plt  
import matplotlib as mpl   

from shapely.geometry import MultiPoint  

import numpy as np  
from scipy.spatial.distance import cdist

In [7]:
#Загрузка границц Невского района
admin_borders = ox.geocode_to_gdf('Адмиралтейский район, Санкт-Петербург')
admin_borders = admin_borders.to_crs("EPSG:4326")

In [9]:
#оценка crs данных
print("CRS округов:", admin_borders.crs)
#проверка данных слоя с границами
admin_borders.head()

CRS округов: epsg:4326


,geometry,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((30.25023 59.90129, 30.25112 59.90103...",30.250229,59.896031,30.342678,59.941057,154959753,relation,1114193,59.918442,30.302795,boundary,administrative,10,0.454171,administrative,Admiralteysky District,"Admiralteysky District, Сенной округ, Saint Pe..."


In [10]:
#загрузка данных школ? больниц и ДОО из OSM
place = "Адмиралтейский район, Санкт-Петербург, Россия"
schools = ox.features_from_place(place, tags={"amenity": "school"})
kids = ox.features_from_place(place, tags={"amenity": "kindergarten"})
hospitals = ox.features_from_place(place, tags={"building": "hospital"})
#формирование центройдов из полигонов
def to_centroids(gdf):
    poly_gdf = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])].copy()
    poly_gdf['geometry'] = poly_gdf.geometry.centroid
    point_gdf = gdf[gdf.geometry.type == 'Point']
    return gpd.GeoDataFrame(pd.concat([poly_gdf, point_gdf]), crs=gdf.crs)
schools_points = to_centroids(schools)
schools_points = schools_points.to_crs("EPSG:32636")
kids_points = to_centroids(kids)
kids_points = kids_points.to_crs("EPSG:32636")
hospitals_points = to_centroids(hospitals)
hospitals_points = hospitals_points.to_crs("EPSG:32636")

# Объединение точек школ? доо и больниц
all_points = gpd.GeoDataFrame(pd.concat([schools_points, kids_points,hospitals_points ]), crs="EPSG:32636")


C:\Users\Tatyana_007\AppData\Local\Temp\ipykernel_15184\3725394551.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poly_gdf['geometry'] = poly_gdf.geometry.centroid
C:\Users\Tatyana_007\AppData\Local\Temp\ipykernel_15184\3725394551.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poly_gdf['geometry'] = poly_gdf.geometry.centroid
C:\Users\Tatyana_007\AppData\Local\Temp\ipykernel_15184\3725394551.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poly_gdf['geometry'] = poly_gdf.geometry.centroid


In [12]:
import folium
from folium.plugins import HeatMap, MarkerCluster, Fullscreen, MiniMap, MeasureControl

# Конвертируем точки в WGS84 (lat/lon)
all_points_wgs84 = all_points.to_crs("EPSG:4326")

# Разделяем точки по типам
schools_wgs84 = schools_points.to_crs("EPSG:4326")
kids_wgs84 = kids_points.to_crs("EPSG:4326")
hospitals_wgs84 = hospitals_points.to_crs("EPSG:4326")

# Центр карты (Адмиралтейский район)
map_center = [59.93, 30.30]

# Создаем базовую карту
m = folium.Map(
    location=map_center,
    zoom_start=14,
    tiles="cartodbpositron",
    control_scale=True
)

# Добавляем полноэкранный режим
Fullscreen(position="topright").add_to(m)

# 1. Слой: Тепловая карта всех объектов
heat_layer = folium.FeatureGroup(name="Тепловая карта")
heat_data_all = [[point.y, point.x] for point in all_points_wgs84.geometry]
HeatMap(heat_data_all, radius=15, blur=20).add_to(heat_layer)
heat_layer.add_to(m)

# 2. Слой: Школы
school_layer = folium.FeatureGroup(name="Школы")
for idx, row in schools_wgs84.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.Icon(icon="graduation-cap", prefix="fa", color="green"),
        popup=f"Школа: {row.get('name', 'Название не указано')}",
        tooltip="Школа"
    ).add_to(school_layer)
school_layer.add_to(m)

# 3. Слой: Детские сады
kids_layer = folium.FeatureGroup(name="Детские сады")
for idx, row in kids_wgs84.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.Icon(icon="child", prefix="fa", color="blue"),
        popup=f"Детский сад: {row.get('name', 'Название не указано')}",
        tooltip="Детский сад"
    ).add_to(kids_layer)
kids_layer.add_to(m)


# 4. Больницы (красные иконки)
hospitals_group = folium.FeatureGroup(name="Больницы", show=True)
for idx, row in hospitals_wgs84.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.Icon(icon="hospital", prefix="fa", color="red"),
        popup=f"<b>Больница:</b> {row['name']}",
        tooltip="Больница"
    ).add_to(hospitals_group)
hospitals_group.add_to(m)


# Добавляем мини-карту
MiniMap(toggle_display=True).add_to(m)

# Добавляем измерение расстояния
MeasureControl(position="bottomleft", primary_length_unit="meters").add_to(m)

# Добавляем управление слоями 
folium.LayerControl(collapsed=False).add_to(m)

# Сохраняем карту
m.save("social_map.html")
m